In [7]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from collections import defaultdict
import nltk
from collections import Counter
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from sklearn.metrics import f1_score

In [8]:


def get_docs_and_labels(file_name):
    """
    :param file_name:
    :return: Return tokenized 2d string matrix
    """
    # Read training documents
    # with codecs.open(file_name, 'r', 'utf-8-sig') as raw_text:
    with open(file_name, 'r') as raw_text:
        # raw_text = raw_text.encode('ascii', 'ignore')
        lines = raw_text.readlines()

    # Tokenization with NLTK
    docs = [nltk.tokenize.word_tokenize(line) for line in lines]
    # docs = [nltk.tokenize.word_tokenize(line.decode('utf-8')) for line in lines]
#     docs = [l.split() for l in lines]

    # remove all tokens that are not alphabetic
    docs = [list(filter(lambda word: word.isalpha(), doc)) for doc in docs if True]

    min_len = 4
    docs = filterLen(docs, min_len)
    
    return [doc[1:] for doc in docs], [doc[:1] for doc in docs]


def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]


def get_tf_idf_matrix(docs):
    """
    :param docs:
    :return: The term frequency - inversed document frequency representation of each document
    """
    tf_matrix = get_tf_matrix(docs)

    scale_with_idf(tf_matrix)

    l2_normalize(tf_matrix)

    return tf_matrix


def get_tf_matrix(docs):
    """
    :param docs:
    :return: The term frequencies of the documents in csr format
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)

    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows + 1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter

    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k, _ in cnt.most_common())
        l = len(keys)
        for j, k in enumerate(keys):
            ind[j + n] = idx[k]
            val[j + n] = cnt[k]
        ptr[i + 1] = ptr[i] + l
        n += l
        i += 1

    tf_matrix = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    tf_matrix.sort_indices()

    return tf_matrix


def scale_with_idf(tf_matrix):
    """
    :param tf_matrix: term frequencies of th corpus
    :return: Scale term frequency with inversed document frequency
    """
    # document word frequency
    df = defaultdict(int)
    for d in docs:
        for w in set(d):
            df[w] += 1

    nrows = tf_matrix.shape[0]
    nnz = tf_matrix.nnz
    ind, val, ptr = tf_matrix.indices, tf_matrix.data, tf_matrix.indptr

    # document frequency
    # df(int)
    for i in ind:
        df[i] += 1

    # inverse document frequency
    for k, v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory

    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]


def l2_normalize(mat):
    """
    Normalize the rows of a CSR matrix by their L-2 norm.
    :param mat:
    """
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0
        for j in range(ptr[i], ptr[i + 1]):
            rsum += val[j] ** 2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0 / np.sqrt(rsum)
        for j in range(ptr[i], ptr[i + 1]):
            val[j] *= rsum


def filter_stop_words(docs, minlen):
    r""" filter out terms that are too short.
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]


def print_csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy,
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
            name, mat.shape[0],
            sum(1 if mat.indptr[i + 1] > mat.indptr[i] else 0
                for i in range(mat.shape[0])),
            mat.shape[1], len(np.unique(mat.indices)),
            len(mat.data)))
    else:
        print("%s: [nrows %d, ncols %d, nnz %d]" % (name, mat.shape[0], mat.shape[1], len(mat.data)))
    pass


def save_tf_idf_matrix_to_disk(tf_idf_matrix):
    return True

#### Main starts here

In [14]:
# ### Main ###

file_name = 'train.dat'

docs, class_labels = get_docs_and_labels(file_name)
print len(docs)

tf_idf_matrix = get_tf_idf_matrix(docs)

14438


In [15]:
### 

#### Find nearest neighbor

In [16]:
def find_nearest_neighbors(training_tf_idf_vector, training_tf_idf_matrix, class_labels, k=1):
    dot_products = training_tf_idf_vector.dot(training_tf_idf_matrix.T)
    sims = list(zip(dot_products.indices, dot_products.data))
    sims.sort(key=lambda x: x[1], reverse=True)
    print sims[:k]
    return [class_labels[s[0]][0] for s in sims[:k] if s[1] > 0]

In [17]:
testing_idx = 3
testing_tf_idf_vector = tf_idf_matrix[testing_idx,:]
print print_csr_info(tf_idf_matrix)
print print_csr_info(testing_tf_idf_vector)
# print("mat2:", tf_idf_matrix[testing_idx, :20].todense(), "\n")
# print("mat2 copy:", testing_tf_idf_vector[:20].todense(), "\n")
print find_nearest_neighbors(testing_tf_idf_vector, tf_idf_matrix, class_labels, k=3)
# TODO: Remove label from training data

: [nrows 14438, ncols 37107, nnz 1060335]
None
: [nrows 1, ncols 37107, nnz 68]
None
[(3, 1.0000000000000004), (13309, 0.37218877664407402), (2254, 0.37218877664407402)]
['Subclavian', 'Correction', 'Correction']


#### Split data into training set and testing set

In [ ]:
### Main ###
docs, class_labels = get_docs_and_labels(file_name)

In [ ]:

docs = docs[0:200]
class_labels = class_labels[0:200]
tf_idf_matrix = get_tf_idf_matrix(docs)

### Validation

In [ ]:
def splitData(mat, cls, fold=1, d=10):
    r""" Split the matrix and class info into train and test data using d-fold hold-out
    """
    n = mat.shape[0]
    r = int(np.ceil(n*1.0/d))
    mattr = []
    clstr = []
    # split mat and cls into d folds
    for f in range(d):
        if f+1 != fold:
            mattr.append( mat[f*r: min((f+1)*r, n)] )
            clstr.extend( cls[f*r: min((f+1)*r, n)] )
    # join all fold matrices that are not the test matrix
    train = sp.vstack(mattr, format='csr')
    # extract the test matrix and class values associated with the test rows
    test = mat[(fold-1)*r: min(fold*r, n), :]
    clste = cls[(fold-1)*r: min(fold*r, n)]

    return train, clstr, test, clste


def classifyNames(tf_idf_matrix, class_labels, k=3, d=10):
    r""" Classify names using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
#     docs = [cmer(n, c) for n in names]
#     mat = build_matrix(docs)
#     # since we're using cosine similarity, normalize the vectors
#     csr_l2normalize(mat)
    
    
    
    def classify(x, train, clstr, k):
        r""" Classify vector x using kNN and majority vote rule given training data and associated classes
        """
        # find nearest neighbors for x
        dots = x.dot(train.T)
        sims = list(zip(dots.indices, dots.data))
        sims.sort(key=lambda x: x[1], reverse=True)
#         print 'k===', k
#         return [clstr[s[0]] for s in sims[:k] if s[1] > 0]
        tc = Counter(clstr[s[0]][0] for s in sims[:k]).most_common(2)
        if len(tc) < 2 or tc[0][1] > tc[1][1]:
            # majority vote
            return tc[0][0]
        
        # tie break
        tc = defaultdict(float)
        for s in sims[:k]:
            tc[clstr[s[0]][0]] += s[1]
        return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]
    
    macc = 0.0
    for f in range(d):
        print '===', f
        # split data into training and testing
        train, clstr, test, clste = splitData(tf_idf_matrix, class_labels, f+1, d)
#         print 'train', clstr[0]
        # predict the class of each test sample
        clspr = [ classify(test[i,:], train, clstr, k) for i in range(test.shape[0]) ]  
#         print 'test', clspr
        # compute the accuracy of the prediction
        acc = 0.0
        for i in range(len(clste)):
            if clste[i][0] == clspr[i]:
                acc += 1
        acc /= len(clste)
        macc += acc
    
#     macc = 0.0
#     for f in range(d):
#         print '===', f
#         # split data into training and testing
#         train, clstr, test, clste = splitData(tf_idf_matrix, class_labels, f+1, d)
# #         print 'train', clstr[0]
#         # predict the class of each test sample
#         clspr = [ classify(test[i,:], train, clstr, k) for i in range(test.shape[0]) ]  
# #         print 'test', clspr
#         # compute the accuracy of the prediction
#         acc = 0.0
#         print f1_score(clste, clspr, average=None)
# #         for i in range(len(clste)):
# #             if clste[i][0] == clspr[i]:
# #                 acc += 1
# #         acc /= len(clste)
# #         macc += acc
        
    return macc/d

In [ ]:
rate = classifyNames(tf_idf_matrix, class_labels, k=5, d=10)
print 'success rate====', rate

In [ ]:
highest_rate = 0
best_c = 0
best_k = 0

# print 'rate. =====', rate
# for c in range(1, 5):
#     print('c=%d' % c)
#     for k in range(1, 7):
#         rate = classifyNames(tf_idf_matrix, class_labels, c, k)
#         if rate > highest_rate:
#             best_c = c
#             best_k = k
#             highest_rate = rate
#         print('\tk=%d, rate=%f' % (k, rate))
# print('best parameters: c=%d, k=%d' % (best_c, best_k))